<a href="https://colab.research.google.com/github/vivekab5/Email-Scraper-Pro/blob/master/Data_cleaning_Vivek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tqdm

In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from tqdm import tqdm
from google.colab import files


from IPython.display import display
uploaded = files.upload()


input_file = list(uploaded.keys())[0]
output_file = "cleaned_websites.csv"


def load_websites(file_path):
    df = pd.read_csv(file_path)
    column = df.columns[0]
    return list(set(df[column].dropna().str.strip().str.lower()))


def check_website(url):
    try:
        if not url.startswith(('http://', 'https://')):
            url = 'http://' + url
        response = requests.get(url, timeout=10)
        if response.status_code < 400:
            return url, "Working"
        else:
            return url, f"Issue (Status: {response.status_code})"
    except requests.exceptions.RequestException as e:
        return url, f"Error ({str(e)})"


def main():
    websites = load_websites(input_file)
    print(f"Checking {len(websites)} websites...\n")

    with ThreadPoolExecutor(max_workers=50) as executor:
        results = list(tqdm(executor.map(check_website, websites), total=len(websites)))


    df_output = pd.DataFrame(results, columns=["Website", "Status"])
    df_output.to_csv(output_file, index=False)
    print(f"\nCompleted. Results saved to '{output_file}'")


    files.download(output_file)


main()
